# Central Pattern Generator for control of quadruped robot

This is part 1 of the Legged robot course's project \#2. We will focus on implementing a montion controller based on a Central Pattern Generato (CPG).

## Importing the required libraries

In [2]:
import time
import numpy as np # numpy library for matrix computatios
from matplotlib import pyplot as plt

from env.quadruped_gym_env import QuadrupedGymEnv
from hopf_network import HopfNetwork

%load_ext autoreload
%autoreload 2

## Note on this notebook vs the hopf_network module

Instead of directly implementing the simulation in the hopf_network module's default function __\_\_main\_\___, we will work it out here in this current notebook. 

This will enable us to test different parameters, perform different graphs, in a reproducible and illustrated step by step fashion. The steps will be explained as needed. 

Finally, we will move the chosen parameters and code to the hopf_network module's default function __\_\_main\_\___, such that it is a standalone module. 

In [3]:
# Initialise dynamic engine of pybullet and we connect it to the Pybullet GUI
ADD_CARTESIAN_PD = True
TIME_STEP = 0.001
foot_y = 0.0838 # this is the hip length 
sideSign = np.array([-1, 1, -1, 1]) # get correct hip sign (body right is negative)

In [ ]:
env = QuadrupedGymEnv(
                  render=True,              # visualize
                  on_rack=False,              # useful for debugging! 
                  isRLGymInterface=False,     # not using RL
                  time_step=TIME_STEP,
                  action_repeat=1,
                  motor_control_mode="TORQUE",
                  add_noise=False,    # start in ideal conditions
                  # record_video=True
                  )

In [ ]:
# After implementing the gait offsets, let's check that everything looks ok (divide by 2pi for clarity)
# initialize Hopf Network, supply gait
print(HopfNetwork(gait="TROT").PHI/(2*np.pi))
print(HopfNetwork(gait="PACE").PHI/(2*np.pi))
print(HopfNetwork(gait="BOUND").PHI/(2*np.pi))
print(HopfNetwork(gait="WALK").PHI/(2*np.pi))

Which seems right !

In [ ]:
# Testing the simulation works, the robot does move according to gravity and small torques...
env.reset()
for i in range(1000):
    env.step(action = np.zeros(12) + 5 ) 

Everything seems to work fine, although there were troubles at the beginning on macOS because of pyqt5.

## Updating the oscillators' states

We will now work on implementing the _integrate_hopf_equations_ method of the HopfNetwork, where we will update the states of the oscillators for each leg : the current aplitudes and phases of each, which we'll later use, mapping them to robot coordinates.

In [ ]:
int_cpg = HopfNetwork(gait="PACE")
int_X = np.array([int_cpg.X])
print(f"\nOsccillator state before : \n{int_cpg.X}")
int_cpg._integrate_hopf_equations()
int_X = np.append(int_X, [int_cpg.X], axis=0)
print(f"\nOsccillator state after one integration : \n{int_cpg.X}")
for i in range(2000):
    int_cpg._integrate_hopf_equations()
    int_X = np.append(int_X, [int_cpg.X], axis=0)
print(f"\nOsccillator state after many integrations : \n{int_cpg.X}")

Let's look at it in plots :

In [ ]:
x = range(int_X.shape[0])

fig, axs = plt.subplots(2, 4, sharex=True, sharey='row')

axs[0, 0].plot(x, int_X[:,0,0])
axs[0, 0].set_title("r0")
axs[0, 1].plot(x, int_X[:,0,1])
axs[0, 1].set_title("r1")
axs[0, 2].plot(x, int_X[:,0,2])
axs[0, 2].set_title("r2")
axs[0, 3].plot(x, int_X[:,0,3])
axs[0, 3].set_title("r3")

axs[1, 0].plot(x, int_X[:,1,0])
axs[1, 0].set_title("θ0")
axs[1, 1].plot(x, int_X[:,1,1])
axs[1, 1].set_title("θ1")
axs[1, 2].plot(x, int_X[:,1,2])
axs[1, 2].set_title("θ2")
axs[1, 3].plot(x, int_X[:,1,3])
axs[1, 3].set_title("θ3");

We can check that indeed the oscillator phases respect the gait timing. This looks ok.

Now, we implement the update function of the HopfNetwork class. We simply map the cpg oscillator states to cartesion coordinates for the feet x and z positions (y are constant).

In [ ]:
int_cpg = HopfNetwork(gait="PACE")
x, z = int_cpg.update()
x_arr = np.array([x])
z_arr = np.array([z])
print(f"\nPositions for first step : \nx: {x_arr} \nz: {z_arr}")
x, z = int_cpg.update()
x_arr = np.append(x_arr, [x], axis=0)
z_arr = np.append(z_arr, [z], axis=0)
print(f"\nPositions after next step : \nx: {x} \nz: {z}")

for i in range(2000):
    x, z = int_cpg.update()
    x_arr = np.append(x_arr, [x], axis=0)
    z_arr = np.append(z_arr, [z], axis=0)
print(f"\nPositions after many steps : \nx: {x} \nz: {z}")

x = range(x_arr.shape[0])

fig, axs = plt.subplots(2, 4, sharex=True, sharey='row')

# Order is FR, FL, RR, RL
axs[0, 0].plot(x, x_arr[:,0])
axs[0, 0].set_title("x FR")
axs[0, 1].plot(x, x_arr[:,1])
axs[0, 1].set_title("x FL")
axs[0, 2].plot(x, x_arr[:,2])
axs[0, 2].set_title("x RR")
axs[0, 3].plot(x, x_arr[:,3])
axs[0, 3].set_title("x RL")

axs[1, 0].plot(x, z_arr[:,0])
axs[1, 0].set_title("z FR")
axs[1, 1].plot(x, z_arr[:,1])
axs[1, 1].set_title("z FL")
axs[1, 2].plot(x, z_arr[:,2])
axs[1, 2].set_title("z RR")
axs[1, 3].plot(x, z_arr[:,3])
axs[1, 3].set_title("z RL");

We can see that the feet positions, in the referential of the robot, are indeed moving as expected, with regular oscillations, with the robot height as 0.25, we can see that the feet touch the ground and "penetrate" about (from -0.25 to -0.26) 0.01, but this is not actual ground penetration, it will only force to push strongly against the ground.

## Motor control

Now let's continue towards actual control. Let's see how motor angles and velocities are :

In [ ]:
print(f"motor angles:\n{env.robot.GetMotorAngles()}")
print(f"motor velocities:\n{env.robot.GetMotorVelocities()}")

From these current angles and velocities, we'll set controls with a PD controller. Let's run a simulation with a few parameters. We can play with these prameters, and visualize what happens.

In [ ]:
# initialize Hopf Network, supply gait
cpg = HopfNetwork(
                mu=1**2,                # converge to sqrt(mu)
                omega_swing=10*2*np.pi,  # MUST EDIT
                omega_stance=1*2*np.pi, # MUST EDIT
#                 gait="TROT",            # change depending on desired gait
#                 gait="PACE",            # change depending on desired gait
                gait="BOUND",            # change depending on desired gait
#                 gait="WALK",            # change depending on desired gait
                coupling_strength=1,    # coefficient to multiply coupling matrix
                couple=True,            # should couple
                time_step=TIME_STEP,    # time step 
                ground_clearance=0.05,  # foot swing height 
                ground_penetration=0.01,# foot stance penetration into ground 
                robot_height=0.25,      # in nominal case (standing) 
                des_step_len=0.04      # desired step length 
                )

simulation_duration = 10 # (in seconds)
TEST_STEPS = int(simulation_duration / (TIME_STEP))
t = np.arange(TEST_STEPS)*TIME_STEP

# [TODO] initialize data structures to save CPG and robot states

############## Sample Gains
# joint PD gains
kp=np.array([150,70,70])
kd=np.array([2,0.5,0.5])
# Cartesian PD gains
kpCartesian = np.diag([2500]*3)
kdCartesian = np.diag([40]*3)

# env._on_rack = True
env._on_rack = False
env.reset()

for j in range(TEST_STEPS):
    # initialize torque array to send to motors
    action = np.zeros(12) 
    # get desired foot positions from CPG 
    xs,zs = cpg.update()
    # [TODO] get current motor angles and velocities for joint PD, see GetMotorAngles(), GetMotorVelocities() in quadruped.py
    q = env.robot.GetMotorAngles()
    dq = env.robot.GetMotorVelocities()
    
    # loop through desired foot positions and calculate torques
    for i in range(4):
        # initialize torques for legi
        tau = np.zeros(3)
        # get desired foot i pos (xi, yi, zi) in leg frame
        leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
        # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
#         leg_q = np.zeros(3) # [TODO] 
        leg_q = env.robot.ComputeInverseKinematics(i, leg_xyz)
        # Add joint PD contribution to tau for leg i (Equation 4)
#         tau += np.zeros(3) # [TODO] 
        tau += kp*(leg_q-q[3*i:3*i+3])+kd*(-dq[3*i:3*i+3]) # assume we want 0 velocity
        
        # add Cartesian PD contribution
        if ADD_CARTESIAN_PD:
            # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
            # [TODO] 
            # Get current foot velocity in leg frame (Equation 2)
            # [TODO] 
            # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
            tau += np.zeros(3) # [TODO]
        
        # Set tau for legi in action vector
        action[3*i:3*i+3] = tau
    
    # send torques to robot and simulate TIME_STEP seconds 
    env.step(action) 
    env.robot.GetBasePosition()
    env.robot._GetDefaultInitPosition()
    if env.is_fallen():
        max_dist = dist;
        break
    
    # [TODO] save any CPG or robot states

With the default parameters, we can see that the robot barely moves around... 

### Grid search for best omega swing/stance

We'll try grid search for the parameters, looking at the best parameters for distance.

In [ ]:
# Test multi dim arrays... 
best_dist = np.zeros(( 4, 5, 1, 3))
print(best_dist.shape)
best_dist[0][0] = [1,2,3]
best_dist

In [5]:
## initialize values that chaange from above

# gridded_gait = "BOUND"         # change depending on desired gait
# gridded_gait="TROT"            # change depending on desired gait
# gridded_gait="PACE"            # change depending on desired gait
gridded_gait="WALK"            # change depending on desired gait

MIN_STANCE_Om = 0.2*np.pi
MAX_Om = 20*2*np.pi
Om_steps_width = 0.2*np.pi
NB_values = int((MAX_Om-MIN_STANCE_Om)/Om_steps_width)
simulation_duration = 10 # (in seconds)

# env.close() # If run mutliple times, need to disconnect first before creating new

# 2d array with 2d for two search axis : first axis is Omega stance, second is Omega swing
distances = np.zeros(( NB_values, NB_values, 1, 3))
Om_stance_range = range(0, NB_values)
Om_swing_range = range(0, NB_values) # omega swing should intuitively be faster than stance, but only do in loop

## Standard code similar to above

env = QuadrupedGymEnv(
                  render=False,              # visualize
                  on_rack=False,              # useful for debugging! 
                  isRLGymInterface=False,     # not using RL
                  time_step=TIME_STEP,
                  action_repeat=1,
                  motor_control_mode="TORQUE",
                  add_noise=False,    # start in ideal conditions
                  # record_video=True
                  )

TEST_STEPS = int(simulation_duration / (TIME_STEP))
t = np.arange(TEST_STEPS)*TIME_STEP

############## Sample Gains
# joint PD gains
kp=np.array([150,70,70])
kd=np.array([2,0.5,0.5])
# Cartesian PD gains
kpCartesian = np.diag([2500]*3)
kdCartesian = np.diag([40]*3)

for i_om_stance in Om_stance_range:
    omega_stance = MIN_STANCE_Om + i_om_stance*Om_steps_width
    print(f"Omega stance={np.round(omega_stance/np.pi,1)}π")
    for i_om_swing in range(i_om_stance, NB_values): 
        omega_swing = MIN_STANCE_Om + i_om_swing*Om_steps_width
        maxval = np.array(distances[:,:,0,0]).ravel()[np.argmax(distances[:,:,0,0])]
        print(f"    Omega swing={omega_swing/np.pi}π ", end="")
        cpg = HopfNetwork(
                mu=1**2,                # converge to sqrt(mu)
                omega_swing=omega_swing,  # MUST EDIT
                omega_stance=omega_stance, # MUST EDIT
                gait=gridded_gait, 
                coupling_strength=1,    # coefficient to multiply coupling matrix
                couple=True,            # should couple
                time_step=TIME_STEP,    # time step 
                ground_clearance=0.05,  # foot swing height 
                ground_penetration=0.01,# foot stance penetration into ground 
                robot_height=0.25,      # in nominal case (standing) 
                des_step_len=0.04,      # desired step length 
                gait_feedback=False
                )
        env.reset()
        for j in range(TEST_STEPS):
            # initialize torque array to send to motors
            action = np.zeros(12) 
            # get desired foot positions from CPG 
            xs,zs = cpg.update()
            # [TODO] get current motor angles and velocities for joint PD, see GetMotorAngles(), GetMotorVelocities() in quadruped.py
            q = env.robot.GetMotorAngles()
            dq = env.robot.GetMotorVelocities()

            # loop through desired foot positions and calculate torques
            for i in range(4):
                # initialize torques for legi
                tau = np.zeros(3)
                # get desired foot i pos (xi, yi, zi) in leg frame
                leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
                # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
        #         leg_q = np.zeros(3) # [TODO] 
                leg_q = env.robot.ComputeInverseKinematics(i, leg_xyz)
                # Add joint PD contribution to tau for leg i (Equation 4)
        #         tau += np.zeros(3) # [TODO] 
                tau += kp*(leg_q-q[3*i:3*i+3])+kd*(-dq[3*i:3*i+3]) # assume we want 0 velocity

                # add Cartesian PD contribution
                if ADD_CARTESIAN_PD:
                    # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
                    # [TODO] 
                    # Get current foot velocity in leg frame (Equation 2)
                    # [TODO] 
                    # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
                    tau += np.zeros(3) # [TODO]

                # Set tau for legi in action vector
                action[3*i:3*i+3] = tau

            # send torques to robot and simulate TIME_STEP seconds 
            env.step(action) 
        #     x, y, z = 
            dxdydz = np.array(env.robot.GetBasePosition()) - np.array(env.robot._GetDefaultInitPosition())
            distances[i_om_stance][i_om_swing] = dxdydz
            
            maxval = np.array(distances[:,:,0,0]).ravel()[np.argmax(distances[:,:,0,0])]
            print(f" with best x dist={np.round(maxval,4)}", end="")
            print("\n", end="")
            if env.is_fallen():
                print("          Robot fell !")
                break

        # [TODO] save any CPG or robot states

# get best values
print(f"Best values for gait={gridded_gait} are :")
x_array = distances[:,:,0,0]
argmax = np.argmax(x_array)
print(f"argmax of x distance={argmax}")
maxval = np.array(x_array).ravel()[np.argmax(x_array)]
print(f"max x distance={maxval}")
best_Om_stan_idx, best_Om_swin_idx = np.unravel_index(argmax, np.array(x_array).shape)
best_Om_stan = MIN_STANCE_Om + best_Om_stan_idx*Om_steps_width
best_Om_swin = MIN_STANCE_Om + best_Om_swin_idx*Om_steps_width
print(f"Corresp best Omega stance={best_Om_stan}")
print(f"Corresp best Omega swing={best_Om_swin}")

Omega stance=0.2π
    Omega swing=0.2π with best x dist before=0.0
    Omega swing=1.2π with best x dist before=0.0
    Omega swing=2.2π with best x dist before=0.0
    Omega swing=3.2π with best x dist before=0.0
    Omega swing=4.2π with best x dist before=0.0
    Omega swing=5.2π with best x dist before=0.0
    Omega swing=6.2π with best x dist before=0.0
    Omega swing=7.2π with best x dist before=0.0
    Omega swing=8.2π with best x dist before=0.0
    Omega swing=9.2π with best x dist before=0.0
    Omega swing=10.2π with best x dist before=0.002977100064847922
    Omega swing=11.2π with best x dist before=0.002977100064847922
    Omega swing=12.200000000000001π with best x dist before=0.002977100064847922
    Omega swing=13.200000000000003π with best x dist before=0.002977100064847922
    Omega swing=14.2π with best x dist before=0.002977100064847922
    Omega swing=15.2π with best x dist before=0.002977100064847922
    Omega swing=16.200000000000003π with best x dist before=0.

KeyboardInterrupt: 

In [7]:
print(f"Best values for gait={gridded_gait} are :")
x_array = distances[:,:,0,0]
argmax = np.argmax(x_array)
print(f"argmax of x distance={argmax}")
maxval = np.array(x_array).ravel()[np.argmax(x_array)]
print(f"max x distance={maxval}")
best_Om_stan_idx, best_Om_swin_idx = np.unravel_index(argmax, np.array(x_array).shape)
best_Om_stan = MIN_STANCE_Om + best_Om_stan_idx*Om_steps_width
best_Om_swin = MIN_STANCE_Om + best_Om_swin_idx*Om_steps_width
print(f"Corresp best Omega stance={np.round(best_Om_stan/np.pi, 2)}π")
print(f"Corresp best Omega swing={np.round(best_Om_swin/np.pi, 2)}π")

Best values for gait=WALK are :
argmax of x distance=167
max x distance=1.8622712877452654
Corresp best Omega stance=8.2π
Corresp best Omega swing=15.2π


Simulate some of the best results we got :

In [7]:
# initialize Hopf Network, supply gait
cpg = HopfNetwork(
                mu=1**2,                # converge to sqrt(mu)
                omega_swing=15.2*np.pi,  # MUST EDIT
                omega_stance=8.2*np.pi, # MUST EDIT
#                 gait="TROT",            # change depending on desired gait
#                 gait="PACE",            # change depending on desired gait
#                 gait="BOUND",            # change depending on desired gait
                gait="WALK",            # change depending on desired gait
                coupling_strength=1,    # coefficient to multiply coupling matrix
                couple=True,            # should couple
                time_step=TIME_STEP,    # time step 
                ground_clearance=0.05,  # foot swing height 
                ground_penetration=0.02,# foot stance penetration into ground 
                robot_height=0.25,      # in nominal case (standing) 
                des_step_len=0.04      # desired step length 
                )

simulation_duration = 10 # (in seconds)
TEST_STEPS = int(simulation_duration / (TIME_STEP))
t = np.arange(TEST_STEPS)*TIME_STEP

# [TODO] initialize data structures to save CPG and robot states

############## Sample Gains
# joint PD gains
kp=np.array([150,70,70])
kd=np.array([2,0.5,0.5])
# Cartesian PD gains
kpCartesian = np.diag([2500]*3)
kdCartesian = np.diag([40]*3)

# env._on_rack = True
env.close()
env = QuadrupedGymEnv(
                  render=True,              # visualize
                  on_rack=False,              # useful for debugging! 
                  isRLGymInterface=False,     # not using RL
                  time_step=TIME_STEP,
                  action_repeat=1,
                  motor_control_mode="TORQUE",
                  add_noise=False,    # start in ideal conditions
                  # record_video=True
                  )

for j in range(TEST_STEPS):
    # initialize torque array to send to motors
    action = np.zeros(12) 
    # get desired foot positions from CPG 
    xs,zs = cpg.update()
    # [TODO] get current motor angles and velocities for joint PD, see GetMotorAngles(), GetMotorVelocities() in quadruped.py
    q = env.robot.GetMotorAngles()
    dq = env.robot.GetMotorVelocities()
    
    # loop through desired foot positions and calculate torques
    for i in range(4):
        # initialize torques for legi
        tau = np.zeros(3)
        # get desired foot i pos (xi, yi, zi) in leg frame
        leg_xyz = np.array([xs[i],sideSign[i] * foot_y,zs[i]])
        # call inverse kinematics to get corresponding joint angles (see ComputeInverseKinematics() in quadruped.py)
#         leg_q = np.zeros(3) # [TODO] 
        leg_q = env.robot.ComputeInverseKinematics(i, leg_xyz)
        # Add joint PD contribution to tau for leg i (Equation 4)
#         tau += np.zeros(3) # [TODO] 
        tau += kp*(leg_q-q[3*i:3*i+3])+kd*(-dq[3*i:3*i+3]) # assume we want 0 velocity
        
        # add Cartesian PD contribution
        if ADD_CARTESIAN_PD:
            # Get current Jacobian and foot position in leg frame (see ComputeJacobianAndPosition() in quadruped.py)
            # [TODO] 
            # Get current foot velocity in leg frame (Equation 2)
            # [TODO] 
            # Calculate torque contribution from Cartesian PD (Equation 5) [Make sure you are using matrix multiplications]
            tau += np.zeros(3) # [TODO]
        
        # Set tau for legi in action vector
        action[3*i:3*i+3] = tau
    
    # send torques to robot and simulate TIME_STEP seconds 
    env.step(action) 
    env.robot.GetBasePosition()
    env.robot._GetDefaultInitPosition()
    if env.is_fallen():
        max_dist = dist;
        break
    
    # [TODO] save any CPG or robot states

WALK


In [ ]:
##################################################### 
# PLOTS
#####################################################
# example
# fig = plt.figure()
# plt.plot(t,joint_pos[1,:], label='FR thigh')
# plt.legend()
# plt.show()

In [ ]:
# time.sleep(2)
pybullet.disconnect()